In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col, year, quarter, when, sum, avg, round, count
import re

In [0]:
%run ../structs/nb_schemas

In [0]:
# ingestion_objects = [
#     'global-super-store-dataset'
# ]

# _schemas = {
#     "global-super-store-dataset": {
#         "schema": StructType([StructField('Row ID', StringType(), True), StructField('Order ID', StringType(), True), StructField('Order Date', StringType(), True), StructField('Ship Date', StringType(), True), StructField('Ship Mode', StringType(), True), StructField('Customer ID', StringType(), True), StructField('Customer Name', StringType(), True), StructField('Segment', StringType(), True), StructField('City', StringType(), True), StructField('State', StringType(), True), StructField('Country', StringType(), True), StructField('Postal Code', StringType(), True), StructField('Market', StringType(), True), StructField('Region', StringType(), True), StructField('Product ID', StringType(), True), StructField('Category', StringType(), True), StructField('Sub-Category', StringType(), True), StructField('Product Name', StringType(), True), StructField('Sales', StringType(), True), StructField('Quantity', StringType(), True), StructField('Discount', StringType(), True), StructField('Profit', StringType(), True), StructField('Shipping Cost', StringType(), True), StructField('Order Priority', StringType(), True)]),
#         "parameters": {
#             "src_api": "kaggle",
#             "src_api_dataset": "apoorvaappz/global-super-store-dataset",
#             "src_file_name_contains": "Global_Superstore2.csv",
#             "src_file_format": "csv",
#             "dest_catalog_name": "silver",
#             "dest_schema_name": "refined",
#             "dest_table_name": "Global_Superstore".lower()
#         },
#     }
# }

# # schema_config = _schemas[dbutils.widgets.get("ingestion_object")]
# schema_config = _schemas
# params = schema_config["global-super-store-dataset"]["parameters"]

# src_catalog_name       = params["dest_catalog_name"]
# src_schema_name        = params["dest_schema_name"]
# src_table_name         = params["dest_table_name"]

# dest_catalog_name       = params["dest_catalog_name"].replace("silver","gold")
# dest_schema_name        = params["dest_schema_name"].replace("refined","analytics")
# dest_table_name         = params["dest_table_name"]
# dest_table_name = dest_table_name + "_"

# # Definir caminho para a camada Gold
# gold_catalog_name = dest_catalog_name #'gold'
# gold_schema_name  = dest_schema_name  #'analytics'
# gold_table_prefix = dest_table_name   #'global_superstore_'

# print()
# print(f"""src_catalog_name:         '{src_catalog_name}'""")
# print(f"""src_schema_name:          '{src_schema_name}'""")
# print(f"""src_table_name:           '{src_table_name}'""")
# print()
# print(f"""dest_catalog_name:        '{gold_catalog_name}'""")
# print(f"""dest_schema_name:         '{gold_schema_name}'""")
# print(f"""dest_table_name:          '{gold_table_prefix}'""")


In [0]:

# spark.sql(f"DROP CATALOG IF EXISTS gold CASCADE")
# spark.sql(f"DROP SCHEMA IF EXISTS gold.analytics CASCADE")

# spark.sql(f"CREATE CATALOG if NOT EXISTS {dest_catalog_name}")
# spark.sql(f"CREATE SCHEMA if NOT EXISTS {dest_catalog_name}.{dest_table_name}")

In [0]:
# Inicializar Spark Session
spark = SparkSession.builder.appName("GlobalSuperstoreAnalysis").getOrCreate()

In [0]:
# Parâmetros de entrada para a tabela Silver
src_catalog_name = 'silver'
src_schema_name = 'refined'
src_table_name = 'global_superstore'

# Parâmetros da camada Gold
gold_catalog_name = 'gold'
gold_schema_name = 'analytics'
gold_table_prefix = 'global_superstore_'

In [0]:
# Caminho completo da tabela Silver
table_path = f"{src_catalog_name}.{src_schema_name}.{src_table_name}"

# Excluir o schema analytics no catálogo gold, se existir
spark.sql(f"DROP SCHEMA IF EXISTS {gold_catalog_name}.{gold_schema_name} CASCADE")

# Criar catálogo gold se não existir
spark.sql(f"CREATE CATALOG IF NOT EXISTS {gold_catalog_name}")

# Criar schema analytics no catálogo gold
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}")

# Carregar os dados da camada Silver
df = spark.read.table(table_path)

In [0]:
# Carregar os dados da camada Silver
df = spark.read.table(table_path)

# Verificar se a coluna order_date existe e é do tipo DateType
if 'order_date' not in df.columns:
    raise ValueError("Coluna 'order_date' não encontrada no DataFrame")
if df.schema['order_date'].dataType.typeName() != 'date':
    raise ValueError("Coluna 'order_date' deve ser do tipo DateType")

Evolução anual das vendas, quantidades vendidas e preço médio por transação. 



In [0]:
# 1. Consulta: Métricas Anuais
df_annual = df.withColumn("year", year(col("order_date"))) \
    .groupBy("year").agg(
        round(sum("sales"), 2).alias("total_sales"),
        sum("quantity").alias("total_quantity"),
        round(avg("sales"), 2).alias("avg_sales")
    ).orderBy("year")
    
df_annual.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}annual_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}annual_metrics criada com sucesso!")


Analisa a sazonalidade das vendas e quantidades vendidas por trimestre, permitindo identificar períodos de maior ou menor desempenho dentro de cada ano.

In [0]:
# 2. Consulta: Métricas Trimestrais
df_quarterly = df.withColumn("year", year(col("order_date"))) \
    .withColumn("quarter", quarter(col("order_date"))) \
    .groupBy("year", "quarter").agg(
        round(sum("sales"), 2).alias("total_sales"),
        sum("quantity").alias("total_quantity")
    ).orderBy("year", "quarter")

df_quarterly.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}quarterly_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}quarterly_metrics criada com sucesso!")

Visão agregada das vendas e quantidades por semestre, útil para análises sazonais em períodos mais amplos que trimestres, mas mais granulares que anos.



In [0]:
# 3. Consulta: Métricas Semestrais
df_semester = df.withColumn("year", year(col("order_date"))) \
    .withColumn("quarter", quarter(col("order_date"))) \
    .withColumn("semester", when(col("quarter").isin(1, 2), "S1").otherwise("S2")) \
    .groupBy("year", "semester").agg(
        round(sum("sales"), 2).alias("total_sales"),
        sum("quantity").alias("total_quantity")
    ).orderBy("year", "semester")

df_semester.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}semester_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}semester_metrics criada com sucesso!")

 Identificar os 5 produtos mais vendidos em termos de receita, além de mostrar suas quantidades vendidas e preço médio, útil para análises de desempenho de produtos.



In [0]:
# 4. Consulta: Métricas por Produto
df_product = df.groupBy("product_name").agg(
    round(sum("sales"), 2).alias("total_sales"),
    sum("quantity").alias("total_quantity"),
    round(avg("sales"), 2).alias("avg_price")
).orderBy(col("total_sales").desc()).limit(5)

df_product.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}product_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}product_metrics criada com sucesso!")


 Identifica os 5 países com maior receita de vendas, além de suas quantidades vendidas e ticket médio, útil para análises de desempenho por mercado geográfico



In [0]:
# 5. Consulta: Métricas por País
df_country = df.groupBy("country").agg(
    round(sum("sales"), 2).alias("total_sales"),
    sum("quantity").alias("total_quantity"),
    round(avg("sales"), 2).alias("avg_ticket")
).orderBy(col("total_sales").desc()).limit(5)

df_country.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}country_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}country_metrics criada com sucesso!")


Analisa o desempenho de vendas por categoria e subcategoria de produtos, permitindo identificar quais segmentos de produtos são mais lucrativos ou populares.



In [0]:
# 6. Consulta: Métricas por Categoria
df_category = df.groupBy("category", "sub-category").agg(
    round(sum("sales"), 2).alias("total_sales"),
    sum("quantity").alias("total_quantity")
).orderBy("category", "sub-category")

df_category.write.mode("overwrite").saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}category_metrics")

print(f"Tabela {gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}category_metrics criada com sucesso!")

In [0]:
# (
#     df
#     .write
#     .format("delta")
#     .mode("overwrite")
#     .saveAsTable(f"{dest_catalog_name}.{dest_database_name}.{dest_table_name}")
# )


In [0]:
# display(
#   spark
#   .table(f"{dest_catalog_name}.{dest_database_name}.{dest_table_name}")
#   .limit(10)
# )

In [0]:
breakpointXXXX